# GRSW observations notebook

This notebook shows how to access the summary table of the observations data compiled for the GRSW workshop.
It require to pull a python package to access easily the data stored on [DACE](https://dace.unige.ch/dashboard/)

First we need to import the function needed for the notebook

In [ ]:
from resonantstate.data_download  import get_metadata_observations, download_observations_samples
from resonantstate.analyse_samples import *
from resonantstate.ell2SFM import *
from resonantstate.simulations_resonance_analysis import *

## Data download

We can download the metatable to have a look on the available observations samples. The table will be returned in the form of a pandas DataFrame (df):

In [ ]:
df_all_obs = get_metadata_observations()
df_all_obs.head()

We can have a look on the available columns:

In [ ]:
df_all_obs.head(0).T

Let's look at the list of analysed systems:

In [ ]:
df_all_obs["star_name"].unique()

Lets keep only the analysis of one of the systems:

In [ ]:
target = "Kepler-128"

df_selected = df_all_obs[df_all_obs["star_name"].isin([target])]
df_selected.head(100)

We can look at the different mass priors used in the analyses:

In [ ]:
df_selected['mass_prior']

Let's download the samples from these systems. If a path is given, it saves the samples into the given directory.

In [ ]:
#download_destination_path = "downloaded_observtions_samples/"
download_destination_path = None
df_list = download_observations_samples(df_selected, download_destination_path)

The downloaded samples are returned as a list. Each element of the list is a dictionnary containing informations on the downloaded sample and the sample itself. 

Looking at the first element of the list, we can see its structure:

In [ ]:
for k,v in df_list[0].items():
    if k == "samples":
        print(f"{k}:\t...")
    else:
        print(f"{k}:\t{v}")

Let's have a look at the samples from the first analysis:

In [ ]:
df = df_list[0]['samples']
df.head()

You can save the dataframe of this sample as a csv file for future analysis:

In [ ]:
#destination_path = 'your_path_to_file/'
#sample_name = df_list[0]['sample_name'] 
#filename = destination_path + sample_name + '.csv'
#df.to_csv(filename, index=False)

Let's say you want to plot a mass-period diagram for a planet. You can directly extract each column from the header of the dataframe. Remember that planet indexing starts from 0.

In [ ]:
n_planet = 0

mass = df[f'mass_planet_star_ratio_{n_planet}']
period = df[f'period_days_{n_planet}']

plt.figure()
plt.scatter(mass, period, alpha=0.1)
plt.xlabel('Mp/M*')
plt.ylabel('P (d)')
plt.title(f'planet {n_planet}')
plt.show()

Now try plotting the mass-eccentricity diagram:

## Analysis part 1: Simple plots

Perhaps you might be more interested in comparing the samples for each planet across different analyses.

We have prepared a few useful functions to make histograms and scatter plots of the different parameters. These functions loop over each analysis in your selected list of dictionaries and creates separate subplots for each planet, allowing you to compare the samples for each planet across different analyses.

You can directly plot the following parameters:
- **"period"**      (in days)
- **"k"**           (ecos(varpi))
- **"h"**           (esin(varpi))
- **"incl"**        (inclination in deg)
- **"omega"**       (longitude of ascending node in deg)
- **"mass"**        (in stellar mass)
- **"radius"**      (in stellar radii)
- **"lambda"**      (mean longitude in deg)

You can also extract the following derived parameters:
- **"ecc"**         (eccentricity)
- **"varpi"**       (longitude of pericenter in deg)
- **"density"**     (in stellar density)

We can have a look at the total of planets included in our list of dictionaries:

In [ ]:
planets = get_all_planets(df_list)
planets

Let's first make histograms comparing the *mass posteriors* for each planet.

In [ ]:
plot_histograms(df_list, param='mass', units='star')

Let's convert the units to something more intuitive.

Valid units are: **"star"** (default), **"sun"**, **"earth"**, **"jup"**, or **"SI"**

Unit conversion applies to parameters *"mass"*, *"radius"*, or *"density"*

Try changing the units in the above histograms.

Let's now make scatter plots of the different parameters. 

In [ ]:
plot_samples(df_list, x_param='mass', y_param='ecc', units='earth')

Try plotting the **period-density** diagrams with the density in **SI units**:

In [ ]:
#plot_samples()

We might also be interested in the posterior distributions of **consecutive planets**:

In [ ]:
plot_adjacent_planets(df_list, 
                      param='mass', 
                      planet_pair=planets[0:2],
                      units='earth')

We can also compare the **period ratios** between adjacent planets:

In [ ]:
compare_period_ratios(df_list, planet_pair=planets[0:2])

Let's now look at the **Kepler-80 system**. Search and download the samples into a list of dictionaries as before.

In [ ]:
target2 = 'Kepler-80'

#df_selected2 = ...

Have a go at visualising the different parameters of the samples as before.

In [ ]:
# your plots here...

### Analysis part 2: Second Fundamental Model of Resonance

For the next part we will be plotting the elliptic elements of one system into the *Second Fundamental Model* of resonance. These functions are intended for planet pairs in **first-order** mean motion resonance. 

Let's start by looking at one set of analysis. You can visualize the available pairs of planets in first-order resonance with the following function:

In [ ]:
df_dict = df_list[2]

plot_ell2SFM(df_dict)   # takes dictionary or dataframe or numpy array

Instead of plotting the samples with a solid colour, you can also give a numpy array of values such as the masses:

In [ ]:
import pandas as pd

samples = df_dict['samples']

total_mass1 = pd.Series.to_numpy(samples['mass_planet_star_ratio_0'] + samples['mass_planet_star_ratio_1'])
min_mass, max_mass = total_mass1.min(), total_mass1.max()

plot_ell2SFM(df_dict, colors=[total_mass1], color_lim=(min_mass, max_mass))

Here is an example visualising more than one pair of planets in a system:

In [ ]:
target3 = "Kepler-60"

df_selected3 = df_all_obs[df_all_obs["star_name"].isin([target3])]
df_list3 = download_observations_samples(df_selected3)

samples_dict3 = df_list3[4]

plot_ell2SFM(samples_dict3)

You can also give numpy arrays such as masses to more than one pair of planets. The colormap scales automatically unless otherwise specified.

In [ ]:
samples3 = samples_dict3['samples']

total_mass1 = pd.Series.to_numpy(samples3['mass_planet_star_ratio_0'] + samples3['mass_planet_star_ratio_1'])
total_mass2 = pd.Series.to_numpy(samples3['mass_planet_star_ratio_1'] + samples3['mass_planet_star_ratio_2'])

plot_ell2SFM(samples_dict3, colors=[total_mass1, total_mass2], color_lim=(None, None))

Use the following space to explore samples from the Kepler-80 dictionaries you downloaded before.

In [ ]:
# your plots here...